In [15]:
import os
from langchain_openai import ChatOpenAI


In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system = """
編輯任務： 從提供的內容中選出大約20句連續且富有故事性的句子。

限制：選句子必須是影片原始內容，不含AI生成文本。
連續性：句子必須完全連續，不跳過任何部分。

"""
# 格式：請依照原格式返回，舉例來說："1\n0:00:00 --> 0:00:05\n跟你旁边的家人说我们都要长大成熟\n\n"。
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_chain = prompt | llm

import concurrent.futures

def ask_question(question):
    ai_message = llm_chain.invoke(question)
    return ai_message.content

def parallel_questions(questions):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        answers = list(executor.map(ask_question, questions))
    return answers

In [17]:
# get mongodb data
from dotenv import load_dotenv
import os
# Load environment variables from a file
load_dotenv('.env')
# Now you can access your environment variables
MONGODB_URI = os.getenv('MONGODB_URI')

In [18]:
from pymongo import MongoClient

client = MongoClient(MONGODB_URI)
db = client["goodder_event"]
collection = db["ai-att-table"]

/Users/michael/miniconda3/envs/goodder_env/lib/python3.9/site-packages/pymongo/uri_parser.py:313: UserWarning: authmechanism must be in ('MONGODB-CR', 'SCRAM-SHA-1', 'MONGODB-X509', 'PLAIN', 'SCRAM-SHA-256', 'MONGODB-OIDC', 'GSSAPI', 'DEFAULT', 'MONGODB-AWS')
  return get_validated_options(opts, warn)


In [19]:
# Find the documents
query = {"full_duration": {"$gt": 4000}}
results = collection.find(query).limit(1)

# Print the documents
for result in results:
    print(result)

{'_id': ObjectId('65e93472a4f75d561300d677'), 'yt_url': 'https://www.youtube.com/watch?v=fVMPP_ooko8', 'yt_id': 'fVMPP_ooko8', 'tstart': None, 'tend': None, 'is_full': True, 'user_id': '641a571d7ad562e6640e9974', 'company_id': '641a57767ad562e6640e9976', 'created_by': '641a571d7ad562e6640e9974', 'updated_by': '641a571d7ad562e6640e9974', 'is_reddemed': True, 'full_duration': 7076, 'output_duration': 7076, 'status': 'SUCCESS', 'updated_at': datetime.datetime(2024, 3, 7, 3, 47, 16, 956000), 'created_at': datetime.datetime(2024, 3, 7, 3, 28, 50, 129000), 'gen_type': 'audio_to_text', 'title': 'PJBC 主日 | 再這樣下去，遲早會餓死 | 黃烱榕牧師 20240225', 'transcript': {'text': '1\n00:00:00,000，-->，00:00:05,000\n跟你旁边的家人说我们都要长大成熟\n\n2\n00:00:05,000，-->，00:00:10,000\n我们一起祷告\n\n3\n00:00:10,000，-->，00:00:12,000\n亲爱的主耶稣\n\n4\n00:00:12,000，-->，00:00:14,000\n帮助我\n\n5\n00:00:14,000，-->，00:00:17,000\n越来越爱你\n\n6\n00:00:17,000，-->，00:00:21,000\n越来越可慕你\n\n7\n00:00:21,000，-->，00:00:24,000\n奉耶稣的名祷告\n\n8\n00:00:24,000，-->，00:0

In [20]:
len(result["transcript"]["text"])

117571

In [21]:
content = result["transcript"]["text"]

In [22]:
content = content.replace("，", " ")

In [23]:
import srt

subtitles = list(srt.parse(content))

In [24]:
for sub in subtitles:
    print(sub)

Subtitle(index=1, start=datetime.timedelta(0), end=datetime.timedelta(seconds=5), content='跟你旁边的家人说我们都要长大成熟', proprietary='')
Subtitle(index=2, start=datetime.timedelta(seconds=5), end=datetime.timedelta(seconds=10), content='我们一起祷告', proprietary='')
Subtitle(index=3, start=datetime.timedelta(seconds=10), end=datetime.timedelta(seconds=12), content='亲爱的主耶稣', proprietary='')
Subtitle(index=4, start=datetime.timedelta(seconds=12), end=datetime.timedelta(seconds=14), content='帮助我', proprietary='')
Subtitle(index=5, start=datetime.timedelta(seconds=14), end=datetime.timedelta(seconds=17), content='越来越爱你', proprietary='')
Subtitle(index=6, start=datetime.timedelta(seconds=17), end=datetime.timedelta(seconds=21), content='越来越可慕你', proprietary='')
Subtitle(index=7, start=datetime.timedelta(seconds=21), end=datetime.timedelta(seconds=24), content='奉耶稣的名祷告', proprietary='')
Subtitle(index=8, start=datetime.timedelta(seconds=24), end=datetime.timedelta(seconds=27), content='一起说', proprietary='')

In [127]:
def split_for_yt_clipping(subtitles, max_length=2700, overlap=20):
    timestamp_content = []
    temp_subs = []
    transcript_slice = ""

    for subtitle in subtitles:
        temp_sub = f"{subtitle.index}\n{subtitle.start} --> {subtitle.end}\n{subtitle.content}\n\n"
        temp_subs.append(temp_sub)
        transcript_slice += temp_sub + " "

        if len(transcript_slice) > max_length:
            timestamp_content.append(''.join(temp_subs))
            transcript_slice = "".join(temp_subs[-overlap:])
            temp_subs = temp_subs[-overlap:]  # Keep the last 'overlap' subtitles for the next slice

    # Add the remaining subtitles if any
    if temp_subs:
        timestamp_content.append(''.join(temp_subs))

    return timestamp_content

def split_transcript(subtitles, max_length=2700):
    transcript = []
    transcript_slice = ""

    for subtitle in subtitles:
        transcript_slice += subtitle.content + " "
        
        if len(transcript_slice) > max_length:
            transcript.append(transcript_slice)
            transcript_slice = ""

    if transcript_slice:
        transcript.append(transcript_slice)
    return transcript

In [128]:
timestamp_content = split_for_yt_clipping(subtitles)

In [130]:
timestamp_content[0]

'1\n0:00:00 --> 0:00:05\n跟你旁边的家人说我们都要长大成熟\n\n2\n0:00:05 --> 0:00:10\n我们一起祷告\n\n3\n0:00:10 --> 0:00:12\n亲爱的主耶稣\n\n4\n0:00:12 --> 0:00:14\n帮助我\n\n5\n0:00:14 --> 0:00:17\n越来越爱你\n\n6\n0:00:17 --> 0:00:21\n越来越可慕你\n\n7\n0:00:21 --> 0:00:24\n奉耶稣的名祷告\n\n8\n0:00:24 --> 0:00:27\n一起说\n\n9\n0:00:27 --> 0:00:29\n阿弥陀佛\n\n10\n0:00:29 --> 0:00:32\n我们拍手欢迎天父耶稣圣灵\n\n11\n0:00:32 --> 0:00:34\n来当我们的灯神\n\n12\n0:00:34 --> 0:00:36\n耶\n\n13\n0:00:36 --> 0:00:38\n祝我们欢迎你\n\n14\n0:00:38 --> 0:00:40\n祝我们欢迎你\n\n15\n0:00:40 --> 0:00:42\n阿弥陀佛\n\n16\n0:00:42 --> 0:00:46\n很开心我们今天可以先用喜乐来敬拜我们的神\n\n17\n0:00:46 --> 0:00:49\n让我们的声音直达天上\n\n18\n0:00:49 --> 0:00:51\n讨天父爸爸的喜悦好不好\n\n19\n0:00:51 --> 0:00:53\n好\n\n20\n0:00:53 --> 0:00:55\n我们来拍手\n\n21\n0:00:57 --> 0:01:02\n阿弥陀佛\n\n22\n0:01:02 --> 0:01:05\n我的盼望\n\n23\n0:01:05 --> 0:01:08\n就在耶稣基督里\n\n24\n0:01:08 --> 0:01:10\n在你里面\n\n25\n0:01:10 --> 0:01:13\n满有彼岸\n\n26\n0:01:13 --> 0:01:15\n阿利路亚\n\n27\n0:01:15 --> 0:01:18\n神明确认\n\n28\n0:01:18 --> 0:01:21\n就在耶稣基督里\n\n29\n0:01:21 --> 0:0

In [131]:
timestamp_content[1]

'64\n0:02:52 --> 0:02:55\n我的盼望\n\n65\n0:02:55 --> 0:02:58\n就在耶稣基督里\n\n66\n0:02:58 --> 0:03:00\n在祢里面\n\n67\n0:03:00 --> 0:03:05\n满有平安\n\n68\n0:03:05 --> 0:03:08\n生命前沿\n\n69\n0:03:08 --> 0:03:11\n就在耶稣基督里\n\n70\n0:03:11 --> 0:03:14\n永驻在我里面\n\n71\n0:03:14 --> 0:03:16\n火车再宣告一次\n\n72\n0:03:16 --> 0:03:18\n我的盼望\n\n73\n0:03:18 --> 0:03:20\n我的盼望\n\n74\n0:03:20 --> 0:03:23\n就在耶稣基督里\n\n75\n0:03:23 --> 0:03:25\n在祢里面\n\n76\n0:03:25 --> 0:03:30\n满有平安\n\n77\n0:03:30 --> 0:03:33\n生命前沿\n\n78\n0:03:33 --> 0:03:36\n就在耶稣基督里\n\n79\n0:03:36 --> 0:03:39\n永驻在我里面\n\n80\n0:03:39 --> 0:03:42\n火车再宣告\n\n81\n0:03:42 --> 0:03:43\n我宣告\n\n82\n0:03:43 --> 0:03:45\n在我里面\n\n83\n0:03:46 --> 0:03:49\n祢是上一线更勇的力量\n\n84\n0:03:49 --> 0:03:51\n我付出这世界\n\n85\n0:03:51 --> 0:03:54\n我靠近耶稣基督\n\n86\n0:03:54 --> 0:03:56\n我宣告\n\n87\n0:03:56 --> 0:03:58\n在我里面\n\n88\n0:03:58 --> 0:04:02\n火车拥有满有能力\n\n89\n0:04:02 --> 0:04:04\n有耶稣在我里面\n\n90\n0:04:04 --> 0:04:06\n活着我勇敢在里\n\n91\n0:04:06 --> 0:04:07\n再一次\n\n92\n0:04:07 --> 0:04:08\n我宣告\n\n93\n0

In [129]:
len(timestamp_content)

46

In [34]:
timestamp_content = []
transcript = []
timestamp_content_slice = ""
transcript_slice = ""

for subtitle in subtitles:
    temp_sub = f"{subtitle.index}\n{subtitle.start} --> {subtitle.end}\n{subtitle.content}\n\n"
    timestamp_content_slice += temp_sub
    transcript_slice += subtitle.content + " "
    
    if len(transcript_slice) > 2700:
        timestamp_content.append(timestamp_content_slice)
        transcript.append(transcript_slice)
        timestamp_content_slice = ""
        transcript_slice = ""

# Append the remaining slices to the lists
if timestamp_content_slice:
    timestamp_content.append(timestamp_content_slice)
if transcript_slice:
    transcript.append(transcript_slice)

In [29]:
result = ask_question(transcript[0])

In [42]:
input = f"""
    時間戳記字幕： {result}
    """

In [43]:
input

'\n    時間戳記字幕： 跟你旁边的家人说我们都要长大成熟。我们一起祷告，亲爱的主耶稣，帮助我，越来越爱你，越来越可慕你。奉耶稣的名祷告。一起说，阿弥陀佛。我们拍手欢迎天父耶稣圣灵来当我们的灯神。耶，祝我们欢迎你。祝我们欢迎你。阿弥陀佛。很开心我们今天可以先用喜乐来敬拜我们的神。让我们的声音直达天上，讨天父爸爸的喜悦好不好？好，我們來拍手。阿弥陀佛。我的盼望就在耶稣基督里，在你里面，满有彼岸。阿利路亚，神明确认，就在耶稣基督里，永驻在我里面活着。我宣告在我里面，祢比世上一切更有能力。我不输在世界，我靠近耶稣基督。我宣告在我里面，有活水涌流满有能力，有耶稣在我里面活着。我勇敢向祢，祝在祢面前，永驻在我里面，没有惧怕能战胜仇敌。永驻在我里面，没有惧怕爱得善忧郁。永驻在我里面，没有惧怕能战胜仇敌。永驻在我里面，没有惧怕爱得善忧郁。来跟你的宝贝，一个掌开拜托，我们的神是能力之源。我们的神是能力之源。大家来跟拜托，来我们用您里面来祷告，然后宣告，不论在什么样困难的环境，来到我们这个正好的环境当中，不要为了要去消逝我们的力量。不要为了要去消逝我们的力量。我们请看护我们，我们愿传承主，祝祢要用我们的力量，祈祷我们，祝祢要用我们的力量，祝祢要用我们的力量。\n    '

In [ ]:
def mapping(timestamp_content, summary):
    input = f"""
    精選內容： {summary}。
    時間戳記內容： {timestamp_content}。
    """
    ai_message = llm_chain.invoke(input)
    return ai_message.content

In [31]:
len(result)

515

test

In [99]:
test = """
1
00:00:00,000 --> 00:00:02,500
和風熏柳，花香醉人，正是南國春光漫爛季節。福建省福州府西門大街，青石板路筆直的伸展出去，直通西門。

2
00:00:03,000 --> 00:00:06,000
一座建構宏偉的宅第之前，左右兩座石壇中各豎一根兩丈來高的旗杆，杆頂飄揚青旗。

3
00:00:07,000 --> 00:00:10,500
右首旗上黃色絲線繡著一頭張牙舞爪、神態威猛的雄獅，旗子隨風招展，顯得雄獅更奕奕若生。雄獅頭頂有一對黑絲線繡的蝙蝠展翅飛翔。左首旗上繡著“福威鏢局”四個黑字，銀鉤鐵划，剛勁非凡。

4
00:00:12,000 --> 00:00:15,000
大宅朱漆大門，門上茶杯大小的銅釘閃閃發光，門頂匾額寫著“福威鏢局”四個金漆大字，下面橫書“總號”兩個小字。

5
00:00:16,000 --> 00:00:20,000
進門處兩排長凳，分坐著八名勁裝結束的漢子，個個腰板筆挺，顯出一股英悍之气。

6
00:00:21,000 --> 00:00:24,500
大門內一片寬闊的天井，地面鋪著整齊的青石板，四周栽種著翠綠的竹子，顯得十分雅致。

7
00:00:25,000 --> 00:00:28,500
天井正中央一口古井，井台上刻著龍飛鳳舞的花紋，井水清澈見底。

8
00:00:29,000 --> 00:00:33,000
福威鏢局的總鏢頭林震南正站在井邊，手持一把銀亮的寶劍，神色凝重。

9
00:00:34,000 --> 00:00:38,000
鏢局內的弟子們正在操練場上練武，刀光劍影，吶喊聲此起彼伏，場面甚是壯觀。

10
00:00:39,000 --> 00:00:42,500
林震南的夫人陳氏站在天井一角，凝望著遠處的天空，似有心事。

11
00:00:43,000 --> 00:00:47,000
忽然，從大門外傳來一陣急促的馬蹄聲，一名信差飛奔而入，滿頭大汗。

12
00:00:48,000 --> 00:00:51,000
信差翻身下馬，急忙走到林震南面前，低聲報告道：“總鏢頭，有急件。”

13
00:00:52,000 --> 00:00:55,000
林震南接過信件，快速拆開，眉頭漸漸皺起。

14
00:00:56,000 --> 00:01:00,000
“快，召集所有鏢師到大堂集合，”林震南果斷下令，聲音中帶著一絲緊張。

15
00:01:01,000 --> 00:01:04,500
不一會兒，大堂內便聚集了二十多名鏢師，每個人都神情肅穆，等待著林震南的指示。

16
00:01:05,000 --> 00:01:08,500
林震南走到大堂中央，將信件高舉過頭，大聲宣讀：“近日江湖上出現一股強盜，專門劫持各大鏢局的貨物，我們福威鏢局也在他們的目標之列。”

17
00:01:09,000 --> 00:01:12,500
眾鏢師聞言，個個義憤填膺，紛紛表示願意誓死保衛鏢局。

18
00:01:13,000 --> 00:01:16,500
林震南揚手示意大家安靜，接著說道：“這次我們要組織一支精英隊伍，務必將這股強盜一網打盡。”

19
00:01:17,000 --> 00:01:21,000
大堂內響起了熱烈的掌聲，所有鏢師都充滿了鬥志和信心。

20
00:01:22,000 --> 00:01:25,500
天井外的竹林中，一隻小鳥驚飛而起，似乎預示著一場激烈的戰鬥即將來臨。

21
00:01:26,000 --> 00:01:30,000
此時，福威鏢局的後院，一個年輕人正在練習劍法，他就是林震南的兒子林平之。

22
00:01:31,000 --> 00:01:34,500
林平之一招一式都十分認真，額頭上已經滿是汗水，但他依然堅持不懈。

23
00:01:35,000 --> 00:01:39,000
忽然，他感覺到一陣異樣的氣息，猛然回頭，看到一名黑衣人悄無聲息地出現在他身後。

24
00:01:40,000 --> 00:01:43,000
林平之驚訝地後退一步，手中的劍已經指向黑衣人，冷聲問道：“你是誰？”

25
00:01:44,000 --> 00:01:48,000
黑衣人冷笑一聲，低聲說道：“我是來取你性命的人。”說罷，黑衣人迅速出手，兩人頓時交鋒在一起。

26
00:01:49,000 --> 00:01:53,000
後院的竹林中，只見劍光閃閃，兩人的身影快速移動，劍氣縱橫。

27
00:01:54,000 --> 00:01:58,000
林平之雖然年輕，但劍法精妙，與黑衣人鬥得難解難分。

28
00:01:59,000 --> 00:02:02,500
忽然，林平之一個漂亮的轉身，劍尖直指黑衣人的咽喉，黑衣人驚慌之下，急忙後退。

29
00:02:03,000 --> 00:02:07,000
林平之穩穩站定，劍指黑衣人，冷冷地說道：“今日放你一馬，下次再敢來犯，必取你性命。”

30
00:02:08,000 --> 00:02:12,000
黑衣人狼狽地退走，林平之鬆了一口氣，收劍入鞘，望著遠去的背影，心中暗暗發誓一定要加倍努力，保衛家族的榮譽。

"""

In [100]:
results = ask_question(test)

In [24]:
import textwrap

def split_content(content, max_length):
    return textwrap.wrap(content, max_length)

# 假設我們想要每段最多 100 個字
segments = split_content(content, 2800)

In [35]:
contents = segments[:3]

In [47]:
results = parallel_questions(contents)

In [55]:
results

[AIMessage(content='和風熏柳，花香醉人，正是南國春光漫爛季節。福建省福州府西門大街，青石板路筆直的伸展出去，直通西門。一座建构宏偉的宅第之前，左右兩座石壇中各豎一根兩丈來高的旗杆，杆頂飄揚青旗。右首旗上黃色絲線繡著一頭張牙舞爪、神態威猛的雄獅，旗子隨風招展，顯得雄獅更奕奕若生。雄獅頭頂有一對黑絲線繡的蝙蝠展翅飛翔。左首旗上繡著“福威鏢局”四個黑字，銀鉤鐵划，剛勁非凡。大宅朱漆大門，門上茶杯大小的銅釘閃閃發光，門頂匾額寫著“福威鏢局”四個金漆大字，下面橫書“總號”兩個小字。進門處兩排長凳，分坐著八名勁裝結束的漢子，個個腰板筆挺，顯出一股英悍之气。突然間后院馬蹄聲響，那八名漢子一齊站起，搶出大門。只見鏢局西側門中沖出五騎馬來，沿著馬道沖到大門之前。當先一匹馬全身雪白，馬勒腳鐙都是爛銀打就，鞍上一個錦衣少年，約莫十八九歲年紀，左肩上停著一頭獵鷹，腰懸寶劍，背負長弓，潑喇喇縱馬疾馳。身后跟隨四騎，騎者一色青布短衣。一行五人馳到鏢局門口，八名漢子中有三個齊聲叫了起來：“少鏢頭又打獵去啦！”那少年哈哈一笑，馬鞭在空中拍的一響，虛擊聲下，胯下白馬昂首長嘶，在青石板大路上沖了出去。一名漢子叫道：“史鏢頭，今儿再抬頭野豬回來，大伙儿好飽餐一頓。”那少年身后一名四十來歲的漢子笑道：“一條野豬尾巴少不了你的，可先別灌飽了黃湯。”眾人大笑聲中，五騎馬早去得遠了。', response_metadata={'token_usage': {'completion_tokens': 919, 'prompt_tokens': 4334, 'total_tokens': 5253}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2974252c-1727-41f6-8a2a-f5af10f7ea8d-0', usage_metadata={'input_tokens': 4334, 'output_tokens': 919, 'total_tokens': 5253}),
 AIMessage(content='林平之气往上沖，伸右手往桌上重重一拍，說道：“甚么東西，兩個不帶眼的狗崽子，卻到我

In [54]:
results[0].content

'和風熏柳，花香醉人，正是南國春光漫爛季節。福建省福州府西門大街，青石板路筆直的伸展出去，直通西門。一座建构宏偉的宅第之前，左右兩座石壇中各豎一根兩丈來高的旗杆，杆頂飄揚青旗。右首旗上黃色絲線繡著一頭張牙舞爪、神態威猛的雄獅，旗子隨風招展，顯得雄獅更奕奕若生。雄獅頭頂有一對黑絲線繡的蝙蝠展翅飛翔。左首旗上繡著“福威鏢局”四個黑字，銀鉤鐵划，剛勁非凡。大宅朱漆大門，門上茶杯大小的銅釘閃閃發光，門頂匾額寫著“福威鏢局”四個金漆大字，下面橫書“總號”兩個小字。進門處兩排長凳，分坐著八名勁裝結束的漢子，個個腰板筆挺，顯出一股英悍之气。突然間后院馬蹄聲響，那八名漢子一齊站起，搶出大門。只見鏢局西側門中沖出五騎馬來，沿著馬道沖到大門之前。當先一匹馬全身雪白，馬勒腳鐙都是爛銀打就，鞍上一個錦衣少年，約莫十八九歲年紀，左肩上停著一頭獵鷹，腰懸寶劍，背負長弓，潑喇喇縱馬疾馳。身后跟隨四騎，騎者一色青布短衣。一行五人馳到鏢局門口，八名漢子中有三個齊聲叫了起來：“少鏢頭又打獵去啦！”那少年哈哈一笑，馬鞭在空中拍的一響，虛擊聲下，胯下白馬昂首長嘶，在青石板大路上沖了出去。一名漢子叫道：“史鏢頭，今儿再抬頭野豬回來，大伙儿好飽餐一頓。”那少年身后一名四十來歲的漢子笑道：“一條野豬尾巴少不了你的，可先別灌飽了黃湯。”眾人大笑聲中，五騎馬早去得遠了。'

In [71]:
content

"1\n00:00:00,000 --> 00:00:02,500\n和風熏柳，花香醉人，正是南國春光漫爛季節。福建省福州府西門大街，青石板路筆直的伸展出去，直通西門。\n\n2\n00:00:03,000 --> 00:00:06,000\n一座建构宏偉的宅第之前，左右兩座石壇中各豎一根兩丈來高的旗杆，杆頂飄揚青旗。\n\n3\n00:00:07,000 --> 00:00:10,500\n右首旗上黃色絲線繡著一頭張牙舞爪、神態威猛的雄獅，旗子隨風招展，顯得雄獅更奕奕若生。雄獅頭頂有一對黑絲線繡的蝙蝠展翅飛翔。左首旗上繡著“福威鏢局”四個黑字，銀鉤鐵划，剛勁非凡。\n\n4\n00:00:12,000 --> 00:00:15,000\n大宅朱漆大門，門上茶杯大小的銅釘閃閃發光，門頂匾額寫著“福威鏢局”四個金漆大字，下面橫書“總號”兩個小字。\n\n\n5\n00:00:16,000 --> 00:00:20,000\n進門處兩排長凳，分坐著八名勁裝結束的漢子，個個腰板筆挺，顯出一股英悍之气。\n\n6\n00:00:21,000 --> 00:00:25,000\nYou can adjust the timing to match your video.\n\n7\n00:00:26,000 --> 00:00:30,000\nMake sure the subtitle text is clear and readable.\n\n8\n00:00:31,000 --> 00:00:35,000\nAnd that's how you create an SRT subtitle file!\n\n9\n00:00:36,000 --> 00:00:40,000\nEnjoy adding subtitles to your videos!"

In [59]:
with open('demo_srt.txt', 'r') as file:
    content = file.read()

segments = split_content(content, 50)

In [65]:
import srt

subtitles = list(srt.parse(content))

for subtitle in subtitles:
    print(subtitle.start, subtitle.end, subtitle.content)

0:00:00 0:00:02.500000 和風熏柳，花香醉人，正是南國春光漫爛季節。福建省福州府西門大街，青石板路筆直的伸展出去，直通西門。
0:00:03 0:00:06 一座建构宏偉的宅第之前，左右兩座石壇中各豎一根兩丈來高的旗杆，杆頂飄揚青旗。
0:00:07 0:00:10.500000 右首旗上黃色絲線繡著一頭張牙舞爪、神態威猛的雄獅，旗子隨風招展，顯得雄獅更奕奕若生。雄獅頭頂有一對黑絲線繡的蝙蝠展翅飛翔。左首旗上繡著“福威鏢局”四個黑字，銀鉤鐵划，剛勁非凡。
0:00:12 0:00:15 大宅朱漆大門，門上茶杯大小的銅釘閃閃發光，門頂匾額寫著“福威鏢局”四個金漆大字，下面橫書“總號”兩個小字。

0:00:16 0:00:20 進門處兩排長凳，分坐著八名勁裝結束的漢子，個個腰板筆挺，顯出一股英悍之气。
0:00:21 0:00:25 You can adjust the timing to match your video.
0:00:26 0:00:30 Make sure the subtitle text is clear and readable.
0:00:31 0:00:35 And that's how you create an SRT subtitle file!
0:00:36 0:00:40 Enjoy adding subtitles to your videos!


In [92]:
len(subtitles[0].content)

49

In [ ]:
max_length = 20
segments = []
for subtitle in subtitles:
    if 
    print(subtitle.content)

In [ ]:
subtitles